<a href="https://colab.research.google.com/github/Nishk23/Federated_Learning/blob/main/fedlearn_training_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow tensorflow-federated

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of tensorflow-federated to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 77.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48

In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input, Add, ReLU
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping,LearningRateScheduler
from tensorflow.keras.regularizers import l2
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from tqdm import tqdm
from tensorflow.keras import datasets
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Splitting the CIFAR-10 dataset among the 3 clients

In [ ]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

# Number of clients
num_clients = 3

# Function to split data among clients
def split_data(x, y, num_clients):
    # Shuffle the data
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)

    # Split the data
    x_shards = np.array_split(x[indices], num_clients)
    y_shards = np.array_split(y[indices], num_clients)

    return x_shards, y_shards

# Split the data
x_train_clients, y_train_clients = split_data(x_train, y_train, num_clients)
x_test_clients, y_test_clients = split_data(x_test, y_test, num_clients)

# Verify the split
for i in range(num_clients):
    print(f"Client {i+1} training data shape: {x_train_clients[i].shape}, labels shape: {y_train_clients[i].shape}")
    print(f"Client {i+1} test data shape: {x_test_clients[i].shape}, labels shape: {y_test_clients[i].shape}")

Client 1 training data shape: (16667, 32, 32, 3), labels shape: (16667, 1)
Client 1 test data shape: (3334, 32, 32, 3), labels shape: (3334, 1)
Client 2 training data shape: (16667, 32, 32, 3), labels shape: (16667, 1)
Client 2 test data shape: (3333, 32, 32, 3), labels shape: (3333, 1)
Client 3 training data shape: (16666, 32, 32, 3), labels shape: (16666, 1)
Client 3 test data shape: (3333, 32, 32, 3), labels shape: (3333, 1)


# Build the Image Classification Model

In [ ]:
# Define the CNN model
def create_cnn_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),

        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Create a model for each client
client_models = [create_cnn_model() for _ in range(num_clients)]

# Summary of the model
client_models[0].summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 2, 2, 64)          0

# Implement the Federated Learning Process

In [ ]:
# Number of rounds of training
num_rounds = 5

# Number of epochs per round for each client
epochs_per_round = 3

# Batch size for training
batch_size = 32

# Function to average the model weights
def federated_averaging(client_weights):
    avg_weights = []
    for weights in zip(*client_weights):
        avg_weights.append(np.mean(weights, axis=0))
    return avg_weights

# Federated learning process with progress bar
for round_num in range(num_rounds):
    print(f"Round {round_num+1}/{num_rounds}")

    # Store client weights after training
    client_weights = []

    # Train each client model on its local data with tqdm progress bar
    for i in tqdm(range(num_clients), desc=f"Training clients for round {round_num+1}"):
        client_models[i].fit(x_train_clients[i], y_train_clients[i],
                             epochs=epochs_per_round, batch_size=batch_size, verbose=0)
        # Append the client model's weights
        client_weights.append(client_models[i].get_weights())

    # Aggregate client weights using federated averaging
    averaged_weights = federated_averaging(client_weights)

    # Set the averaged weights to all client models
    for i in range(num_clients):
        client_models[i].set_weights(averaged_weights)

    print("Model weights have been averaged and updated across all clients.\n")

print("Federated Learning process completed.")

Round 1/5


Training clients for round 1: 100%|██████████| 3/3 [00:32<00:00, 10.95s/it]


Model weights have been averaged and updated across all clients.

Round 2/5


Training clients for round 2: 100%|██████████| 3/3 [00:30<00:00, 10.03s/it]


Model weights have been averaged and updated across all clients.

Round 3/5


...

Model weights have been averaged and updated across all clients.

Round 4/5


Training clients for round 4: 100%|██████████| 3/3 [00:30<00:00, 10.00s/it]


Model weights have been averaged and updated across all clients.

Round 5/5


Training clients for round 5: 100%|██████████| 3/3 [00:29<00:00,  9.99s/it]

Model weights have been averaged and updated across all clients.

Federated Learning process completed.


# Evaluate and Store the Performance Metrics

In [ ]:
# Specify the path where the JSON file should be saved
save_path = '/content/drive/My Drive/Federated_Learning'
os.makedirs(save_path, exist_ok=True)  # Create the directory if it doesn't exist

In [ ]:
# Evaluate the model on each client's test data and store metrics
performance_metrics = {}

for i in range(num_clients):
    print(f"Evaluating client {i+1}")

    # Predict on test data
    y_pred = np.argmax(client_models[i].predict(x_test_clients[i]), axis=1)
    y_true = y_test_clients[i].flatten()

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    # Store metrics in dictionary
    performance_metrics[f'client_{i+1}'] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

    print(f"Client {i+1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

# Save the performance metrics to a JSON file in the specified location
json_file_path = os.path.join(save_path, 'simple_CNN_client_performance_metrics.json')
with open(json_file_path, 'w') as f:
    json.dump(performance_metrics, f, indent=4)

print(f"Performance metrics have been saved to '{json_file_path}'")


Evaluating client 1
105/105 [==============================] - 0s 3ms/step
Client 1 - Accuracy: 0.5987, Precision: 0.5992, Recall: 0.5996, F1 Score: 0.5917
Evaluating client 2
105/105 [==============================] - 0s 3ms/step
Client 2 - Accuracy: 0.5977, Precision: 0.5972, Recall: 0.5990, F1 Score: 0.5904
Evaluating client 3
105/105 [==============================] - 0s 3ms/step
Client 3 - Accuracy: 0.5815, Precision: 0.5808, Recall: 0.5790, F1 Score: 0.5738
Performance metrics have been saved to '/content/drive/My Drive/Federated_Learning/simple_CNN_client_performance_metrics.json'


# Model Improvement

In [ ]:
# Hyperparameters
num_rounds = 5
epochs_per_round = 10
batch_size = 64
learning_rate = 0.001
l2_lambda = 0.001
dropout_rate = 0.5

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Increase Model Complexity with Regularization

In [ ]:
def create_complex_cnn_model():
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3), kernel_regularizer=l2(l2_lambda)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(dropout_rate),

        Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(l2_lambda)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(dropout_rate),

        Conv2D(256, (3, 3), activation='relu', kernel_regularizer=l2(l2_lambda)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(dropout_rate),

        Flatten(),
        Dense(512, activation='relu', kernel_regularizer=l2(l2_lambda)),
        BatchNormalization(),
        Dropout(dropout_rate),

        Dense(10, activation='softmax')
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
# Create models for each client
client_models = [create_complex_cnn_model() for _ in range(num_clients)]

client_models[0].summary()

# Learning Rate Scheduling and Early Stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 30, 30, 64)        1792      
                                                                 
 batch_normalization_12 (Ba  (None, 30, 30, 64)        256       
 tchNormalization)                                               
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 15, 15, 64)        0         
 ng2D)                                                           
                                                                 
 dropout_15 (Dropout)        (None, 15, 15, 64)        0         
                                                                 
 conv2d_19 (Conv2D)          (None, 13, 13, 128)       73856     
                                                                 
 batch_normalization_13 (Ba  (None, 13, 13, 128)      

# Federated Learning Process with Enhanced Aggregation

In [ ]:
def federated_averaging(client_weights):
    avg_weights = []
    for weights in zip(*client_weights):
        avg_weights.append(np.mean(weights, axis=0))
    return avg_weights

# Federated Learning with Progress Bar
for round_num in range(num_rounds):
    print(f"Round {round_num+1}/{num_rounds}")

    client_weights = []
    for i in tqdm(range(num_clients), desc=f"Training clients for round {round_num+1}"):
        model = client_models[i]
        # Train the model with augmented data
        model.fit(datagen.flow(x_train_clients[i], y_train_clients[i], batch_size=batch_size),
                  epochs=epochs_per_round,
                  validation_data=(x_test_clients[i], y_test_clients[i]),
                  callbacks=[reduce_lr, early_stop],
                  verbose=0)

        # Save the trained weights
        client_weights.append(model.get_weights())

    # Federated Averaging with the client weights
    averaged_weights = federated_averaging(client_weights)

    # Update the models with the averaged weights
    for model in client_models:
        model.set_weights(averaged_weights)

    print("Model weights have been averaged and updated across all clients.\n")

print("Federated Learning process completed.")

Round 1/5


Training clients for round 1:   0%|          | 0/3 [00:00<?, ?it/s]


Epoch 10: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.


Training clients for round 1: 100%|██████████| 3/3 [06:42<00:00, 134.24s/it]


Model weights have been averaged and updated across all clients.

Round 2/5


...


Epoch 5: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
Restoring model weights from the end of the best epoch: 2.
Epoch 7: early stopping


Training clients for round 5:  67%|██████▋   | 2/3 [02:52<01:27, 87.38s/it]


Epoch 6: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
Restoring model weights from the end of the best epoch: 3.
Epoch 8: early stopping


Training clients for round 5: 100%|██████████| 3/3 [04:38<00:00, 92.75s/it]

Model weights have been averaged and updated across all clients.

Federated Learning process completed.


# Evaluate the model on each client's test data and store metrics

In [ ]:
performance_metrics = {}

for i in range(num_clients):
    print(f"Evaluating client {i+1}")

    y_pred = np.argmax(client_models[i].predict(x_test_clients[i]), axis=1)
    y_true = y_test_clients[i].flatten()

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    performance_metrics[f'client_{i+1}'] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

    print(f"Client {i+1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

# Save the performance metrics to a JSON file
json_file_path = os.path.join(save_path, 'complex_CNN_client_performance_metrics.json')
with open(json_file_path, 'w') as f:
    json.dump(performance_metrics, f, indent=4)

print(f"Performance metrics have been saved to '{json_file_path}'")

Evaluating client 1
105/105 [==============================] - 1s 7ms/step
Client 1 - Accuracy: 0.7340, Precision: 0.7437, Recall: 0.7344, F1 Score: 0.7269
Evaluating client 2
105/105 [==============================] - 1s 7ms/step
Client 2 - Accuracy: 0.7177, Precision: 0.7256, Recall: 0.7181, F1 Score: 0.7083
Evaluating client 3
105/105 [==============================] - 1s 7ms/step
Client 3 - Accuracy: 0.7327, Precision: 0.7466, Recall: 0.7318, F1 Score: 0.7248
Performance metrics have been saved to '/content/drive/My Drive/Federated_Learning/comples_CNN_client_performance_metrics.json'


# Improvising the model

In [ ]:
# Hyperparameters
num_rounds = 5
epochs_per_round = 10
batch_size = 64
learning_rate = 0.001
l2_lambda = 0.001
dropout_rate = 0.5
num_clients = 3

# Advanced Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

# Function to encode string labels
def encode_labels(y_train, y_test):
    label_encoder = LabelEncoder()
    one_hot_encoder = OneHotEncoder(sparse=False)

    y_train_int = label_encoder.fit_transform(y_train)
    y_test_int = label_encoder.transform(y_test)

    y_train_oh = one_hot_encoder.fit_transform(y_train_int.reshape(-1, 1))
    y_test_oh = one_hot_encoder.transform(y_test_int.reshape(-1, 1))

    return y_train_oh, y_test_oh, label_encoder

# Enhanced Model with Residual Connections and Label Smoothing

In [ ]:
def residual_block(x, filters, kernel_size=(3, 3), activation='relu'):
    shortcut = x
    x = Conv2D(filters, kernel_size, padding='same', activation=activation, kernel_regularizer=l2(l2_lambda))(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters, kernel_size, padding='same', activation=None, kernel_regularizer=l2(l2_lambda))(x)
    x = BatchNormalization()(x)

    # Adjust the shape of the shortcut to match the output if necessary
    if shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, (1, 1), padding='same', kernel_regularizer=l2(l2_lambda))(shortcut)
        shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = ReLU()(x)  # Add ReLU after the addition
    x = Dropout(dropout_rate)(x)
    return x

def create_resnet_model():
    inputs = Input(shape=(32, 32, 3))

    x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(l2_lambda))(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    x = residual_block(x, 64)
    x = MaxPooling2D((2, 2))(x)

    x = residual_block(x, 128)
    x = MaxPooling2D((2, 2))(x)

    x = residual_block(x, 256)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    x = Dense(512, activation='relu', kernel_regularizer=l2(l2_lambda))(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)

    outputs = Dense(10, activation='softmax')(x)

    model = Model(inputs, outputs)

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
                  metrics=['accuracy'])

    return model

In [ ]:
# Create models for each client
client_models = [create_resnet_model() for _ in range(num_clients)]

client_models[0].summary()

# Learning Rate Scheduling and Early Stopping
def one_cycle_scheduler(epoch, max_lr=learning_rate):
    return max_lr * (1 - epoch / (epochs_per_round * num_rounds))

lr_scheduler = LearningRateScheduler(one_cycle_scheduler)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_13 (InputLayer)       [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d_108 (Conv2D)         (None, 32, 32, 64)           1792      ['input_13[0][0]']            
                                                                                                  
 batch_normalization_120 (B  (None, 32, 32, 64)           256       ['conv2d_108[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 max_pooling2d_48 (MaxPooli  (None, 16, 16, 64)           0         ['batch_normalization_1

# Federated Learning Process with Enhanced Aggregation

In [ ]:
def federated_averaging(client_weights):
    avg_weights = []
    for weights in zip(*client_weights):
        avg_weights.append(np.mean(weights, axis=0))
    return avg_weights

# Assuming x_train_clients and y_train_clients are defined and contain the data for each client
for i in range(num_clients):
    y_train_clients[i], y_test_clients[i], label_encoder = encode_labels(y_train_clients[i], y_test_clients[i])

# Federated Learning with Progress Bar and Error Handling
for round_num in range(num_rounds):
    print(f"Round {round_num+1}/{num_rounds}")
    try:
        client_weights = []
        for i in tqdm(range(num_clients), desc=f"Training clients for round {round_num+1}"):
            model = client_models[i]
            # Train the model with augmented data
            model.fit(datagen.flow(x_train_clients[i], y_train_clients[i], batch_size=batch_size),
                      epochs=epochs_per_round,
                      validation_data=(x_test_clients[i], y_test_clients[i]),
                      callbacks=[lr_scheduler, early_stop],
                      verbose=0)

            # Save the trained weights
            client_weights.append(model.get_weights())

        # Federated Averaging with the client weights
        averaged_weights = federated_averaging(client_weights)

        # Update the models with the averaged weights
        for model in client_models:
            model.set_weights(averaged_weights)

        print("Model weights have been averaged and updated across all clients.\n")

    except Exception as e:
        print(f"An error occurred during round {round_num+1}: {str(e)}")
        continue

print("Federated Learning process completed.")

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please chang

Round 1/5


Training clients for round 1: 100%|██████████| 3/3 [10:57<00:00, 219.04s/it]


Model weights have been averaged and updated across all clients.

Round 2/5


Training clients for round 2: 100%|██████████| 3/3 [10:48<00:00, 216.07s/it]


...

Model weights have been averaged and updated across all clients.

Round 4/5


Training clients for round 4: 100%|██████████| 3/3 [10:45<00:00, 215.33s/it]


Model weights have been averaged and updated across all clients.

Round 5/5


Training clients for round 5: 100%|██████████| 3/3 [10:47<00:00, 215.72s/it]

Model weights have been averaged and updated across all clients.

Federated Learning process completed.


# Evaluate the model on each client's test data and store metrics

In [ ]:
# Evaluate the model on each client's test data and store metrics
performance_metrics = {}

for i in range(num_clients):
    print(f"Evaluating client {i+1}")
    try:
        # Generate predictions
        y_pred = np.argmax(client_models[i].predict(x_test_clients[i]), axis=1)
        y_true = np.argmax(y_test_clients[i], axis=1)

        # Debugging Info: Print shapes of y_pred and y_true
        # print(f"Client {i+1}: y_pred shape: {y_pred.shape}, y_true shape: {y_true.shape}")

        # Check for matching number of samples
        if len(y_pred) != len(y_true):
            raise ValueError(f"Inconsistent number of samples: y_pred has {len(y_pred)}, y_true has {len(y_true)}")

        # Calculate metrics
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, average='macro')
        recall = recall_score(y_true, y_pred, average='macro')
        f1 = f1_score(y_true, y_pred, average='macro')

        performance_metrics[f'client_{i+1}'] = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1
        }

        print(f"Client {i+1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    except Exception as e:
        print(f"An error occurred during evaluation of client {i+1}: {str(e)}")
        continue

# Save the performance metrics to a JSON file
save_path = '/content/drive/My Drive/Federated_Learning'
os.makedirs(save_path, exist_ok=True)
json_file_path = os.path.join(save_path, 'enhanced_resnet_client_performance_metrics.json')
with open(json_file_path, 'w') as f:
    json.dump(performance_metrics, f, indent=4)

print(f"Performance metrics have been saved to '{json_file_path}'")


Evaluating client 1
105/105 [==============================] - 1s 13ms/step
Client 1 - Accuracy: 0.8053, Precision: 0.8234, Recall: 0.8087, F1 Score: 0.8038
Evaluating client 2
105/105 [==============================] - 1s 13ms/step
Client 2 - Accuracy: 0.8026, Precision: 0.8230, Recall: 0.8007, F1 Score: 0.7977
Evaluating client 3
105/105 [==============================] - 2s 15ms/step
Client 3 - Accuracy: 0.8098, Precision: 0.8290, Recall: 0.8088, F1 Score: 0.8062
Performance metrics have been saved to '/content/drive/My Drive/Federated_Learning/enhanced_resnet_client_performance_metrics.json'
